In [ ]:
! pip install osmnx
! pip install python-igraph
! apt-get install libcairo2-dev libjpeg-dev libgif-dev
! pip install pycairo

In [ ]:
import numpy as np
import networkx as nx
import igraph as ig
import osmnx as ox

In [ ]:
place_name = "Oricola, L'Aquila, Italia"

# Assuming an average walking speed of 5 km/h
avg_walking_speed = 5 # km/h

In [ ]:
# Get the walk graph from place name
G = ox.graph_from_place(place_name, simplify=False, network_type="walk")

# Calculate edge travel times in seconds
for u, v, k, data in G.edges(data=True, keys=True):
    data['travel_time'] = (data['length']/1000)/(avg_walking_speed/3600)

In [ ]:
out_network = ig.Graph()

In [ ]:
# Build a simple graph where edges can be traversed in both directions (u, v) and (v, u)
added_vertices = []
for u, v, k, data in G.edges(data=True, keys=True):

    # Get start - end nodes
    node_u = G.nodes[u]
    node_v = G.nodes[v]

    # Convert osmid to string to avoid "long integer too large for conversion to C int" error
    u_str = str(u)
    v_str = str(v)
    edge_osmid = str(data["osmid"])

    # Check if vertex u already exist
    if u_str not in added_vertices:    
        # Add Vertex u to graph
        out_network.add_vertex(u_str,
                               osmid=u_str,
                               x=node_u["x"],
                               y=node_u["y"])
        added_vertices.append(u_str)

    # Check if vertex v already exist
    if v_str not in added_vertices:
        # Add Vertex v to graph
        out_network.add_vertex(v_str,
                               osmid=v_str,
                               x=node_v["x"],
                               y=node_v["y"])
        added_vertices.append(v_str)

    # Add edge u-v
    out_network.add_edge(u_str,
                         v_str,
                         osmid=edge_osmid,
                         from_node=u_str,
                         to_node=v_str,
                         name_it=data.get("name", None),
                         highway=data.get("highway", None),
                         length=data.get("length", None),
                         travel_time=data.get("travel_time", None))

In [ ]:
print(f"Number of nodes: {out_network.vcount()}")
print(f"Number of edges: {out_network.ecount()}")

Number of nodes: 3588
Number of edges: 7458


In [ ]:
# Get the first node
out_network.vs[0]

igraph.Vertex(<igraph.Graph object at 0x7fd4ef606050>, 0, {'osmid': '294217498', 'x': 13.0608018, 'y': 42.0828683, 'name': '294217498'})

In [ ]:
# Get the first edge
out_network.vs[0]

igraph.Vertex(<igraph.Graph object at 0x7fd4ef606050>, 0, {'osmid': '294217498', 'x': 13.0608018, 'y': 42.0828683, 'name': '294217498'})

In [ ]:
# Get "294217498" vertex using find
out_network.vs.find(osmid = "294217498")

igraph.Vertex(<igraph.Graph object at 0x7fd4ef606050>, 0, {'osmid': '294217498', 'x': 13.0608018, 'y': 42.0828683, 'name': '294217498'})

In [ ]:
# List of vertices as DataFrame
vertices_df = out_network.get_vertex_dataframe()
vertices_df.head()

,osmid,x,y,name
vertex ID,,,,
0,294217498,13.060802,42.082868,294217498
1,6138209553,13.060635,42.082704,6138209553
2,294217499,13.060413,42.082485,294217499
3,6138209558,13.059905,42.081938,6138209558
4,307868601,13.056502,42.078190,307868601


In [ ]:
# List of edges as DataFrame
edges_df = out_network.get_edge_dataframe()
edges_df.head()

,source,target,osmid,from_node,to_node,name_it,highway,length,travel_time
edge ID,,,,,,,,,
0,0,1,238553826,294217498,6138209553,Strada Statale 5 Via Tiburtina Valeria,primary,22.890,16.48080
1,2,3,238553826,294217499,6138209558,Strada Statale 5 Via Tiburtina Valeria,primary,73.927,53.22744
2,1,2,238553826,294217499,6138209553,Strada Statale 5 Via Tiburtina Valeria,primary,30.438,21.91536
3,4,5,365729329,307868601,3697335306,Strada Statale 5 Via Tiburtina Valeria,primary,8.722,6.27984
4,4,6,365729329,307868601,2054854604,Strada Statale 5 Via Tiburtina Valeria,primary,55.225,39.76200


In [ ]:
# Shortest path from vertex "6135875028" to "3076648512"
node_start = out_network.vs.find(name="6135875028")
node_end = out_network.vs.find(name="3076648512")
route = out_network.get_shortest_paths(node_start.index, node_end.index, "travel_time")

# Get route edges
route_edges = [out_network.get_eid(o1, o2) for o1, o2 in zip(route[0], route[0][1:])]

In [ ]:
# Calculate travel time (in minutes) and route length (meters)
travel_time = sum([out_network.es[edge]["travel_time"] for edge in route_edges])/60
route_m = sum([out_network.es[edge]["length"] for edge in route_edges])

print(f"Avg walking speed: {avg_walking_speed} km/h")
print(f"Travel time: {round(travel_time)}'")
print(f"Length: {round(route_m,2)} m")

Avg walking speed: 5 km/h
Travel time: 8'
Length: 663.87 m


In [ ]:
# Networkx performances
%%timeit
nx.shortest_path(G, 6135875028, 3076648512, "travel_time")

1000 loops, best of 5: 1.14 ms per loop


In [ ]:
# Better performance for igraph
%%timeit
out_network.get_shortest_paths(node_start.index, node_end.index, "travel_time")

1000 loops, best of 5: 274 µs per loop


In [ ]:
# Plot the graph and the route
layout = out_network.layout_auto()
layout.mirror(1)

ig.plot(out_network,
        layout=layout,
        bbox=(0, 0, 500, 500),
        vertex_label=["Start" if vertex.index == node_start.index else 
                      "End" if vertex.index == node_end.index else "" for vertex in out_network.vs],
        vertex_label_dist=[2 if vertex.index == node_start.index else 
                           -6 if vertex.index == node_end.index else 0 for vertex in out_network.vs],
        vertex_label_size=10,
        vertex_size=[5 if vertex.index in (node_start.index, node_end.index) else 0 for vertex in out_network.vs],
        vertex_color=["red" if vertex.index in [node_start.index, node_end.index] else "black" for vertex in out_network.vs],
        edge_curved=0,      
        edge_width=[1 if (edge.index in route_edges) else 0.1 for edge in out_network.es],   
        edge_color=["red" if (edge.index in route_edges) else "black" for edge in out_network.es])

Output hidden; open in https://colab.research.google.com to view.